In [2]:
from pcaf_env import *
from functools import reduce
import matplotlib.pyplot as plot
import pycountry

# define source and destination tables
# LULUCF (Land Use, Land-Use Change and Forestry)

ingest_table = 'sf_scope_results'
src_table_1 = 'sf_oecd_imgr_fco2'
models_table = 'pcaf_dbt_models'

sql=f"""
select country_iso_code,partner_iso_code,industry_code,validity_date,attribute,value,value_units from {ingest_catalog}.{ingest_schema}.{src_table_1}_source """ 
df1 = pd.read_sql(sql, engine)
df1
#df1["units"] = "kt"
#df1 = requantify_df(df1).convert_dtypes()
#df1.info(verbose=True)
#df1


2023-04-07 15:31:52,916 [26726] WARNING  pint.util: Redefining 'C' (<class 'pint.delegates.txt_defparser.plain.UnitDefinition'>)
2023-04-07 15:31:52,935 [26726] WARNING  pint.util: Redefining 'N' (<class 'pint.delegates.txt_defparser.plain.UnitDefinition'>)
2023-04-07 15:31:52,949 [26726] WARNING  pint.util: Redefining 'NOX' (<class 'pint.delegates.txt_defparser.plain.UnitDefinition'>)
2023-04-07 15:31:52,951 [26726] WARNING  pint.util: Redefining 'gNOX' (<class 'pint.delegates.txt_defparser.plain.UnitDefinition'>)
2023-04-07 15:31:52,952 [26726] WARNING  pint.util: Redefining 'tNOX' (<class 'pint.delegates.txt_defparser.plain.UnitDefinition'>)
2023-04-07 15:31:52,960 [26726] WARNING  pint.util: Redefining 'S' (<class 'pint.delegates.txt_defparser.plain.UnitDefinition'>)
2023-04-07 15:31:54,000 [26726] WARNING  pint.util: Redefining 'a' (<class 'pint.delegates.txt_defparser.plain.UnitDefinition'>)
2023-04-07 15:31:54,000 [26726] WARNING  pint.util: Redefining 'yr' (<class 'pint.delegat

using connect string: trino://MichaelTiemannOSC@trino-secure-odh-trino.apps.odh-cl2.apps.os-climate.org:443/osc_datacommons_dev
create schema if not exists osc_datacommons_dev.mdt_sandbox


,country_iso_code,partner_iso_code,industry_code,validity_date,attribute,value,value_units
0,RUS,WLD,DTOTAL,1995,Foreign CO2 emissions embodied in gross imports,73.052,Mt CO2
1,RUS,WLD,DTOTAL,1996,Foreign CO2 emissions embodied in gross imports,79.276,Mt CO2
2,RUS,WLD,DTOTAL,1997,Foreign CO2 emissions embodied in gross imports,81.665,Mt CO2
3,RUS,WLD,DTOTAL,1998,Foreign CO2 emissions embodied in gross imports,58.464,Mt CO2
4,RUS,WLD,DTOTAL,1999,Foreign CO2 emissions embodied in gross imports,47.648,Mt CO2
...,...,...,...,...,...,...,...
334651,LVA,MMR,D35,2014,Foreign CO2 emissions embodied in gross imports,0.000,Mt CO2
334652,LVA,MMR,D35,2015,Foreign CO2 emissions embodied in gross imports,0.000,Mt CO2
334653,LVA,MMR,D35,2016,Foreign CO2 emissions embodied in gross imports,0.000,Mt CO2
334654,LVA,MMR,D35,2017,Foreign CO2 emissions embodied in gross imports,0.000,Mt CO2


<font size=3> Remove invalid country codes </font>

In [7]:
df_country = pd.DataFrame([ country.alpha_3 for country in pycountry.countries ] + ['ROW'], columns=['country_iso_code'], dtype='string')
df_country.info(verbose=True)
#df1.info(verbose=True)
df1 = pd.merge(df1,df_country,on=['country_iso_code'])
df1=df1.convert_dtypes()
df1

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 1 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   country_iso_code  250 non-null    string
dtypes: string(1)
memory usage: 2.1 KB


,country_iso_code,partner_iso_code,industry_code,validity_date,attribute,value,value_units
0,RUS,WLD,DTOTAL,1995,Foreign CO2 emissions embodied in gross imports,73.052,Mt CO2
1,RUS,WLD,DTOTAL,1996,Foreign CO2 emissions embodied in gross imports,79.276,Mt CO2
2,RUS,WLD,DTOTAL,1997,Foreign CO2 emissions embodied in gross imports,81.665,Mt CO2
3,RUS,WLD,DTOTAL,1998,Foreign CO2 emissions embodied in gross imports,58.464,Mt CO2
4,RUS,WLD,DTOTAL,1999,Foreign CO2 emissions embodied in gross imports,47.648,Mt CO2
...,...,...,...,...,...,...,...
270139,LVA,MMR,D35,2014,Foreign CO2 emissions embodied in gross imports,0.0,Mt CO2
270140,LVA,MMR,D35,2015,Foreign CO2 emissions embodied in gross imports,0.0,Mt CO2
270141,LVA,MMR,D35,2016,Foreign CO2 emissions embodied in gross imports,0.0,Mt CO2
270142,LVA,MMR,D35,2017,Foreign CO2 emissions embodied in gross imports,0.0,Mt CO2


Calcuate Scope 2:
GHG emissions occurring as a consequence of the domestic use of grid-supplied electricity, heat,
steam and/or cooling which is imported from another territory


In [3]:
rename_columns = {'value':'total','partner_iso_code':'country_iso_code','value_units' :'total_units'}
agg_columns = { 'value' : 'sum'}
columns_order = ['attribute','industry_code','country_iso_code','validity_date','total','total_units']
df1 = (df1.groupby(['partner_iso_code','industry_code','attribute','validity_date','value_units'],as_index=False)
       .agg(agg_columns)
       .rename(columns=rename_columns)
       .reindex(columns=columns_order)
       .convert_dtypes())

df1.info(verbose=True)
df1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4032 entries, 0 to 4031
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   attribute         4032 non-null   string 
 1   industry_code     4032 non-null   string 
 2   country_iso_code  4032 non-null   string 
 3   validity_date     4032 non-null   Int64  
 4   total             4032 non-null   Float64
 5   total_units       4032 non-null   string 
dtypes: Float64(1), Int64(1), string(4)
memory usage: 197.0 KB


,attribute,industry_code,country_iso_code,validity_date,total,total_units
0,Foreign CO2 emissions embodied in gross imports,D35,APEC,1995,39.478,Mt CO2
1,Foreign CO2 emissions embodied in gross imports,D35,APEC,1996,39.637,Mt CO2
2,Foreign CO2 emissions embodied in gross imports,D35,APEC,1997,46.108,Mt CO2
3,Foreign CO2 emissions embodied in gross imports,D35,APEC,1998,47.416,Mt CO2
4,Foreign CO2 emissions embodied in gross imports,D35,APEC,1999,47.128,Mt CO2
...,...,...,...,...,...,...
4027,Foreign CO2 emissions embodied in gross imports,DTOTAL,ZSCA,2014,209.262,Mt CO2
4028,Foreign CO2 emissions embodied in gross imports,DTOTAL,ZSCA,2015,214.719,Mt CO2
4029,Foreign CO2 emissions embodied in gross imports,DTOTAL,ZSCA,2016,206.428,Mt CO2
4030,Foreign CO2 emissions embodied in gross imports,DTOTAL,ZSCA,2017,209.894,Mt CO2


In [4]:
# divide the dataframe into two dataframes 
# to calculate Scope2 and Scope3 Values
df_D35 = df1[df1['industry_code']=='D35']
df_D35 = df_D35[['attribute','country_iso_code','validity_date','total','total_units']]
df_DTOTAL = df1[df1['industry_code']=='DTOTAL']
df_DTOTAL = df_DTOTAL[['country_iso_code','validity_date','total','total_units']]
df_result = pd.merge(df_D35,df_DTOTAL,on=['country_iso_code','validity_date'])  
df_result=df_result.convert_dtypes()
#df_result.drop(columns=['attribute_y','total_units_y'],inplace=True)
rename_columns = {'total_x':'scope2_value','total_y':'scope3_value','total_units_x':'scope2_value_units','total_units_y':'scope3_value_units'}
df_result.rename(columns=rename_columns,inplace=True)
df_result['scope3_value'] = df_result['scope3_value'] - df_result['scope2_value']
df_result[df_result['country_iso_code']=='CAN']

,attribute,country_iso_code,validity_date,scope2_value,scope2_value_units,scope3_value,scope3_value_units
216,Foreign CO2 emissions embodied in gross imports,CAN,1995,5.129,Mt CO2,179.111,Mt CO2
217,Foreign CO2 emissions embodied in gross imports,CAN,1996,5.269,Mt CO2,188.198,Mt CO2
218,Foreign CO2 emissions embodied in gross imports,CAN,1997,6.362,Mt CO2,202.972,Mt CO2
219,Foreign CO2 emissions embodied in gross imports,CAN,1998,7.993,Mt CO2,214.108,Mt CO2
220,Foreign CO2 emissions embodied in gross imports,CAN,1999,9.1,Mt CO2,222.868,Mt CO2
221,Foreign CO2 emissions embodied in gross imports,CAN,2000,18.191,Mt CO2,236.036,Mt CO2
222,Foreign CO2 emissions embodied in gross imports,CAN,2001,19.531,Mt CO2,228.887,Mt CO2
223,Foreign CO2 emissions embodied in gross imports,CAN,2002,8.195,Mt CO2,233.5,Mt CO2
224,Foreign CO2 emissions embodied in gross imports,CAN,2003,8.22,Mt CO2,232.392,Mt CO2
225,Foreign CO2 emissions embodied in gross imports,CAN,2004,8.411,Mt CO2,238.375,Mt CO2
